In [1]:
import gpytorch

In [96]:
import torch
import gpytorch

class GPFunctionWithOutliers:
    def __init__(self, kernel_params, kernel_type='matern', outlier_ratio=0.1):
        """
        Initializes the GP model with the specified kernel and outlier ratio.

        Args:
            kernel_params (dict): Dictionary containing kernel hyperparameters.
                Expected keys are 'outputscale', 'lengthscale', and 'outlier_noise_scale'.
            kernel_type (str): Type of kernel to use ('matern' or 'rational_quadratic').
            outlier_ratio (float): Probability that a point will be treated as an outlier.
        """
        self.kernel_params = kernel_params
        self.kernel_type = kernel_type
        self.outlier_ratio = outlier_ratio
        
        # Initialize Gaussian likelihood
        self.likelihood = gpytorch.likelihoods.GaussianLikelihood()

    def _build_model(self, x):
        """
        Internal method to build the GP model.
        
        Args:
            x (Tensor): The input x-values for the model.
            
        Returns:
            GPModel: A GP model using the specified kernel type and parameters.
        """
        model = GPModel(train_x=x, train_y=torch.zeros_like(x), likelihood=self.likelihood, kernel_type=self.kernel_type)
        model.covar_module.outputscale = torch.tensor(self.kernel_params['outputscale'])
        model.covar_module.base_kernel.lengthscale = torch.tensor(self.kernel_params['lengthscale'])
        return model

    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        """
        Generates a GP sample path with outliers based on the outlier ratio.

        Args:
            x (Tensor): Input tensor of shape (N,) or (N, 1) where N is the number of data points.

        Returns:
            Tensor: Output tensor of shape (N, 1) representing the GP sample path with possible outliers.
        """
        # Ensure x is of shape (N, 1)
        if x.ndim == 1:
            x = x.unsqueeze(1)
        elif x.ndim == 2 and x.shape[1] == 1:
            pass
        else:
            raise ValueError("Input must be of shape (N,) or (N, 1)")

        # Build and evaluate the model
        model = self._build_model(x)
        model.eval()
        self.likelihood.eval()

        # Generate the sample path from the GP posterior
        with torch.no_grad():
            observed_pred = self.likelihood(model(x))
            sample_path = observed_pred.mean

        # # Apply outliers based on outlier ratio
        # for i in range(sample_path.size(0)):
        #     if torch.rand(1).item() < self.outlier_ratio:
        #         sample_path[i] += (torch.rand(1) - 0.5) * self.kernel_params.get('outlier_noise_scale', 1.0)

        return sample_path


class GPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel_type='matern'):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        
        # Use the specified kernel type
        if kernel_type == 'matern':
            self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5))
        elif kernel_type == 'rational_quadratic':
            self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RationalQuadraticKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Example usage:
if __name__ == "__main__":
    # Define kernel parameters
    kernel_params = {
        'outputscale': 1.0,  # Output scale
        'lengthscale': 1.0,  # Length scale
        'outlier_noise_scale': 3.0  # Noise scale for outliers
    }

    # Instantiate the GP function with outliers
    gp_function = GPFunctionWithOutliers(kernel_params, kernel_type='matern', outlier_ratio=0.1)

    # Example input
    x = torch.linspace(-5, 5, 100)

    # Generate the sample path
    sample_path = gp_function(x)

    # Print or plot the result (e.g., using matplotlib)
    print(sample_path)


RuntimeError: Flattening the training labels failed. The most common cause of this error is that the shapes of the prior mean and the training labels are mismatched. The shape of the train targets is torch.Size([100, 1]), while the reported shape of the mean is torch.Size([100]).

In [97]:
import gpytorch
import torch

# ガウス過程モデルの定義
class GPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.RBFKernel()
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Likelihoodを設定
likelihood = gpytorch.likelihoods.GaussianLikelihood()

# 訓練データなしでインスタンス化
model = GPModel(None, None, likelihood)


In [99]:
model.forward(torch.tensor([1.0])).mean

tensor([0.], grad_fn=<ExpandBackward0>)